# Road Following（道路に沿った走行 - データの収集)
このノートブックでは、カメラ画像を読み込み、JetBotが目指すべき道路上のポイントをクリックしてデータを収集します。  
通常、ディープラーニングの学習データは画像ファイルとラベルファイルに分かれています。  
カメラ画像はJpegファイルとして保存します。ラベルはクリックした画像の座標x, y値になりますが、ここではラベルファイルとして保存するのではなく、Jpegファイル名にx, yの値を付けて保存することにします。

collision avoidanceを実行していれば、今回実行する次の3ステップには馴染みがあるでしょう。

1. データ収集
2. 学習
3. 走行

collision avoidanceでは、画像に対して2つのラベル「free」、「blocked」のどちらに該当するかを判断することができました。画像を2クラスに分類しているため、これを画像分類と呼びます。  
road followingでも、データ収集から走行までは同じような流れで実行していきます。  
ただし、今回はJetBotが道路（または任意のパスや通路、コース）を走行できるようにするために、画像分類の代わりに、**regression（回帰）**という別の手法で学習させます。

このノートブックでの作業は簡単です。

 1. JetBotを道路上のさまざまな位置に配置します（道路中央から離れた位置や、さまざまな方向など）。
> ポイントは、JetBotが正常に走行するルートの他に、ルートから外れそうな所で正常なルートに戻すためのデータを取ることです。  
> データの多様性こそが重要であると、collision avoidanceのときに覚えたはずです。

2. JetBotのカメラ画像を画面に表示します。
3. マウスで画像をクリックすることで、`緑の点`が追加された画像が表示されます。
4. クリックすると画像と一緒にこの`緑の点`のX、Y座標が保存されます。

ここでの重要な点は何でしょうか？ 下記が役立つと思われるガイドです。

1. カメラのライブ映像を見ます。
2. JetBotがたどるべき経路を想像します。
3. JetBotが道路をふらつかずにまっすぐ進むことができるように、目標点はこの道路に沿ったできるだけ遠くの位置に配置します。
> 例として、もし真っ直ぐの道路があったとします。目標点は地平線にします。もし道路が鋭く曲がったカーブなら、脱輪しないレベルでJetBotの近くに配置する必要があるでしょう。

4. JetBotが道路から外れてしまいそうなときに、道路に戻すようなデータも取ります。
 
ディープラーニングのモデルを意図したとおりに機能させるために、十分なデータ量を取得します。  
意図したとおりに動作するモデルでは、以下の動作が期待できます。

1. JetBotは、目標点に向かって安定して移動できます（道路の外に出ることなく）。
2. 道路に沿って設定した目標点を目指して継続的に処理されます。

`02_train_model_JP.ipynb`による学習では、集めたデータを基にX, Yの値の推論をおこなうモデルの学習をします。  
`04_live_demo_trt_JP.ipynb`による自動走行では、予測されたX, Yの値からJetBotのステアリングの値を計算します。(角度は「完全に正確」ではありません。本来であればカメラレンズの歪みを補正するための画像キャリブレーションが必要になりますが、JetBotの動作は角度にほぼ比例するため、キャリブレーション無しでもJetBotの制御は正常に機能します)

### ラベル付けのデモ動画

下記URLの動画を参考に、画像にラベルを付ける方法の例を確認します。このモデルはたった123枚の画像で動作しました:)

YouTubeのサンプル動画：[https://www.youtube.com/embed/FW4En6LejhI](https://www.youtube.com/embed/FW4En6LejhI)

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

## ライブラリの読み込み
それでは、「データ収集」の目的で必要なすべてのライブラリをインポートすることから始めましょう。 OpenCVを使用して画像をみながらラベル付けをします。画像ファイルの命名にはクリックした座標のX, Y値の他に、ファイル名が重複しないようにuuidライブラリも使用されます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import ipywidgets  # Jupyter標準のウィジェットを利用します。
import traitlets  # カメラ画像などのデータが更新されたときに、連動して処理を実行させるためにtraitletsライブラリを利用します。
import ipywidgets.widgets as widgets  # Jupyter標準のウィジェットを利用します。
from IPython.display import display  # ウィジェットを表示するためのdisplayライブラリを利用します。

# JetBot用に用意したカメラ、画像変換、モーター制御ライブラリを利用します。
from jetbot import Robot, Camera, bgr8_to_jpeg

########################################
# 画像注釈のために必要なPythonの基本的なライブラリを読み込みます。
########################################
from uuid import uuid1
import os  # ディレクトリ作成のためにpython標準のosライブラリを利用します。
import json  # jsonは使っていないので省略可能です。
import glob  # 画像ファイル一覧取得のためのライブラリを利用します。
import datetime  # 日付を取得するためのライブラリを利用します。（これはデータセットをzipファイルに圧縮する際に使われていますが、この日本語翻訳ノートブックでは不要のため削除してあります。）
import numpy as np  # 数値計算ライブラリのnumpyを利用します。
import cv2  # OpenCVライブラリを利用します。
import time  # timeは使っていないので省略可能です。

## データ収集

クリック可能なカメラ画像を表示するウィジェットと、クリックした目標地点を画像にオーバーレイして表示するウィジェットを作成します。  
今回はクリックしたX, Y座標を取得できるようにNVIDIAのjaybdubさんが作った[jupyter_clickable_image_widget](https://github.com/jaybdub/jupyter_clickable_image_widget)を利用します。これによりカメラ画面をクリックするだけでデータ収集が可能になります。

JetBotのCameraクラスを使用して、CSI MIPIカメラを有効にします。私たちのニューラルネットワークは、224x224ピクセルの画像を入力として受け取ります。データセットのファイルサイズを最小化するために、カメラをそのサイズに設定します。JetBotはこの画像サイズで機能することを確認しています。  
実施するシナリオによっては、もっと大きな画像サイズでデータを収集し、後で目的のサイズに縮小する方がよい場合があります。

次のコードブロックには、左側にクリックできるカメラ映像ウィジェットが表示され、右側にはクリック後に目標地点が緑の丸で表示された画像のスナップショットのウィジェットが表示されます。
その下には、保存した画像の数が表示されます。

左側のライブ画像をクリックすると、「dataset_xy」ディレクトリにカメラ画像をjpegデータとして保存します。  
その時のファイル名にはターゲットの座標x,yとuuidをが含まれています。  

``xy_<x value>_<y value>_<uuid>.jpg``  

学習時に、ファイル名からx,yの値を復元し、jpeg画像とともに読み込みます。

実行時に`jupyter lab build`するようにポップアップが表示されることがあります。  
これはターミナルで`sudo jupyter lab build`を実行すると解決します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from jupyter_clickable_image_widget import ClickableImageWidget  # 画像をクリックした点のx,y座標を取得するJupyter用ウィジェットを利用します。

DATASET_DIR = 'dataset_xy'  # データ保存ディレクトリ名を定義します。

########################################
# データセット保存用のディレクトリを作成します。
# ディレクトリがすでに存在する場合、ディレクトリ作成関数がエラーを返す可能性があるため、
# ここは「try/except」ステートメントで囲みます。
########################################
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('ディレクトリが存在しているため、作成をスキップします。')

########################################
# カメラを有効化します。
########################################
camera = Camera(fps=4)

########################################
# カメラ画像表示用のウィジェットを用意します。
########################################
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)

########################################
# スナップショット画像表示用のウィジェットを用意します。
########################################
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)

########################################
# traitletsライブラリを利用してカメラ画像データが更新されたときに、
# bgr8フォーマットをjpegフォーマットに変換してから
# 画像表示ウィジェットに反映するように設定します。
########################################
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

########################################
# 保存済みの画像ファイル数を表示するテキストボックスを作成します。
########################################
count_widget = ipywidgets.IntText(description='count')

########################################
# 保存済みの画像ファイル数を数えてウィジェットの値を初期化します。
########################################
count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

########################################
# カメラ画像ウィジェットが持つjpeg画像データを
# 指定されたディレクトリにjpgファイルとして保存します。
########################################
def save_snapshot(_, content, msg):
    ####################
    # jupyter_clickable_image_widgetによってクリックイベントが検出された時に実行します。
    ####################
    if content['event'] == 'click':
        data = content['eventData']  # jupyter_clickable_image_widgetによって提供されたイベント発生時のデータを取得します。
        x = data['offsetX']  # jupyter_clickable_image_widgetによって提供されたクリックした画像のx座標を取得します。
        y = data['offsetY']  # jupyter_clickable_image_widgetによって提供されたクリックした画像のy座標を取得します。

        ####################
        # ピクセル座標をパーセンテージに変換します。
        # これは学習時のget_x()とget_y()関数に合わせた修正になります。
        ####################
        x_ratio = int((x/224)*100)
        y_ratio = int((y/224)*100)
        
        ####################
        # 保存するファイル名を決定します。
        ####################
        uuid = 'xy_%03d_%03d_%s' % (x_ratio, y_ratio, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')

        ####################
        # カメラ画像ウィジェットが持つ画像データをファイルに保存します。
        ####################
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)
        
        ####################
        # スナップショットウィジェットにカメラ画像とクリックした場所を緑の丸で表示します。
        ####################
        snapshot = camera.value.copy()  # OpenCVカメラクラスが持つ画像データを、変数snapshotにコピーします。（ここでスナップショットに表示する画像はクリック時のカメラ画像ウィジェットのデータではないことに注意。ウィジェットが持つのはjpegデータなのでOpenCVで緑の丸を描くためにはbgrに変換する必要が発生します。）
        snapshot = cv2.circle(snapshot, (int(x), int(y)), 8, (0, 255, 0), 3)  # クリックした座標に緑の丸を半径8ピクセル、太さ3ピクセルで描きます。
        snapshot_widget.value = bgr8_to_jpeg(snapshot)  # OpenCV BGR画像をJpeg画像に変換してスナップショットウィジェットを更新します。
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))  # 保存済み画像ファイルを数え直して、カウントウィジェットを更新します。
        
########################################
# jupyter_clickable_image_widgetで作られた
# カメラウィジェットが（クリック）イベントを
# 検出したときにsave_snapshot()関数が呼ばれるように
# 紐づけます。
########################################
camera_widget.on_msg(save_snapshot)

########################################
# カメラ画像ウィジェットとスナップショットウィジェット
# および保存件数表示ウィジェットをひとまとめにした
# data_collection_widgetを作成します。
########################################
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    count_widget
])

########################################
# data_collection_widgetを表示します。
########################################
display(data_collection_widget)

# カメラの終了
データを取り終えたら、他のノートブックでカメラを使用できるようにカメラの接続を適切に閉じましょう。

In [ ]:
camera.stop()  # カメラを停止します。

## 次
十分なデータを収集したら、トレーニングに進みます。  
JetBot本体で学習する場合は、このノートブックを閉じてからJupyter左側にある「Running Terminals and Kernels」を選択して「01_data_collection_JP.ipynb」の横にある「SHUT DOWN」をクリックしてJupyter Kernelをシャットダウンしてから[02_train_model_JP.ipynb](02_train_model_JP.ipynb)に進んでください。  